<a href="https://colab.research.google.com/github/victoruwazurike1/Hamoye_Internship_Notebooks/blob/main/classification_tag_along.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier 
from sklearn.model_selection import RandomizedSearchCV

In [20]:
df = pd.read_csv('energy_data.csv')

In [21]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [23]:
df['stabf'].value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [24]:
df = df.drop(columns = 'stab')

In [25]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,unstable


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tau1    10000 non-null  float64
 1   tau2    10000 non-null  float64
 2   tau3    10000 non-null  float64
 3   tau4    10000 non-null  float64
 4   p1      10000 non-null  float64
 5   p2      10000 non-null  float64
 6   p3      10000 non-null  float64
 7   p4      10000 non-null  float64
 8   g1      10000 non-null  float64
 9   g2      10000 non-null  float64
 10  g3      10000 non-null  float64
 11  g4      10000 non-null  float64
 12  stabf   10000 non-null  object 
dtypes: float64(12), object(1)
memory usage: 1015.8+ KB


In [27]:
df['stabf'] = df['stabf'].astype('category')

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   tau1    10000 non-null  float64 
 1   tau2    10000 non-null  float64 
 2   tau3    10000 non-null  float64 
 3   tau4    10000 non-null  float64 
 4   p1      10000 non-null  float64 
 5   p2      10000 non-null  float64 
 6   p3      10000 non-null  float64 
 7   p4      10000 non-null  float64 
 8   g1      10000 non-null  float64 
 9   g2      10000 non-null  float64 
 10  g3      10000 non-null  float64 
 11  g4      10000 non-null  float64 
 12  stabf   10000 non-null  category
dtypes: category(1), float64(12)
memory usage: 947.5 KB


In [29]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,unstable


In [30]:
X = df.drop(columns = 'stabf')
y = df['stabf']

In [31]:
x_train, x_test, y_train, y_test = train_test_split(X, y , test_size = 0.2, random_state = 1)

In [32]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.fit_transform(x_test)


In [33]:
rcf = RandomForestClassifier(random_state = 1)
rcf.fit(x_train_scaled, y_train)
etc = ExtraTreesClassifier(random_state=1)
etc.fit(x_train_scaled, y_train)

ExtraTreesClassifier(random_state=1)

In [34]:
rcf_predictions = rcf.predict(x_test_scaled)
etc_predictions = etc.predict(x_test_scaled)

In [35]:
rcf_accuracy = accuracy_score(rcf_predictions, y_test)
print(rcf_accuracy)
etc_accuracy = accuracy_score(etc_predictions, y_test)
print(etc_accuracy)

0.928
0.926


In [36]:
xgb = XGBClassifier(random_state=1)
xgb.fit(x_train_scaled, y_train)
xgb_predictions = xgb.predict(x_test_scaled)
print(xgb_predictions)

['unstable' 'unstable' 'stable' ... 'stable' 'unstable' 'unstable']


In [37]:
xgb_accuracy = accuracy_score(xgb_predictions, y_test)
xgb_accuracy

0.919

In [38]:
lgb = LGBMClassifier(random_state=1)
lgb.fit(x_train_scaled, y_train)
lgb_predictions = lgb.predict(x_test_scaled)
lgb_accuracy = accuracy_score(lgb_predictions, y_test)
print(lgb_accuracy)

0.9355


In [ ]:
etc_random_search = ExtraTreesClassifier(random_state=1)
# clf = RandomizedSearchCV(etc_random_search, tr)